In [1]:
!pip install pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 36.2 MB/s eta 0:00:00


Getting data

In [50]:
# Download the Robbins-Pathologic2005.txt dataset
!wget https://github.com/Nihar1402-iit/LLM/blob/main/Robbins-Pathologic2005.txt -O Robbins-Pathologic2005.txt


--2024-07-26 13:15:18--  https://github.com/Nihar1402-iit/LLM/blob/main/Robbins-Pathologic2005.txt
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘Robbins-Pathologic2005.txt’

Robbins-Pathologic2     [  <=>               ] 162.18K   669KB/s    in 0.2s    

2024-07-26 13:15:19 (669 KB/s) - ‘Robbins-Pathologic2005.txt’ saved [166069]



In [56]:
# read it in to inspect it
with open('Robbins-Pathologic2005.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [57]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  2291963


In [58]:
# let's look at the first 1000 characters
print(text[2000:3000])

he patient, to
determine dosages and the best treatment for each individual patient. Neither the publisher nor the authors assume any liability for any injury and/or damage to persons or property arising
from this publication.

Previous editions copyrighted 1999, 1994, 1989, 1984, 1979, 1974
Library of Congress Cataloging-in-Publication Data
Robbins and Cotran pathologic basis of disease.—7th ed./[edited by]
Vinay Kumar, Abul K. Abbas, Nelson Fausto ; with illustrations by James A. Perkins. p. ; cm.
Rev. ed. of: Robbins pathologic basis of disease, 1999.
ISBN 0-7216-0187-1
1. Pathology.
[DNLM: 1. Pathology. QZ 4 R6354 2004] I. Title: Pathologic basis of disease. II. Kumar, Vinay. III. Abbas, Abul K. IV. Fausto, Nelson. V. Robbins, Stanley L. (Stanley Leonard). VI. Cotran,
Ramzi S. Robbins pathologic basis of disease.
RB111.R62 2004
616.07—dc22
2004046835
Publishing Director: William Schmitt
Managing Editor: Rebecca Gruliow
Design Manager: Ellen Zanolle
Printed in China
Last digit is t

Identify unique characters and print them along with the vocab size

In [60]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !"$%&'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[]abcdefghijklmnopqrstuvwxyz§©°±µ½×Üáéïöúüˆαβγδζκλ–—†‡•™→∆∼≤≥■●
123


# Create Character to Integer Mapping and Encoding/Decoding Functions

In [61]:

# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))

[69, 70, 70, 2, 81, 69, 66, 79, 66]
hii there


# let's now encode the entire text dataset and store it into a torch.Tensor

In [62]:

import torch # we use PyTorch: https://pytorch.org
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000]) # the 1000 characters we looked at earier will to the GPT look like this

torch.Size([2291963]) torch.int64
tensor([42,  0,  0, 42, 42,  0,  0, 42, 42, 42,  0,  0, 51, 76, 63, 63, 70, 75,
        80,  2, 62, 75, 65,  2, 36, 76, 81, 79, 62, 75,  2, 49, 34, 53, 41, 48,
        45, 48, 40, 42, 36,  2, 35, 34, 52, 42, 52,  2, 48, 39,  2, 37, 42, 52,
        38, 34, 52, 38,  0, 52, 66, 83, 66, 75, 81, 69,  2, 38, 65, 70, 81, 70,
        76, 75,  0,  0, 55, 42, 47, 34, 58,  2, 44, 54, 46, 34, 51,  2, 46, 35,
        35, 52, 13,  2, 46, 37, 13,  2, 39, 51, 36, 49, 62, 81, 69,  0, 34, 73,
        70, 64, 66,  2, 41, 76, 68, 68, 66,  2, 62, 75, 65,  2, 34, 79, 81, 69,
        82, 79,  2, 35, 62, 66, 79,  2, 49, 79, 76, 67, 66, 80, 80, 76, 79,  0,
        36, 69, 62, 70, 79, 74, 62, 75, 13,  0, 37, 66, 77, 62, 79, 81, 74, 66,
        75, 81,  2, 76, 67,  2, 49, 62, 81, 69, 76, 73, 76, 68, 86,  0, 53, 69,
        66,  2, 54, 75, 70, 83, 66, 79, 80, 70, 81, 86,  2, 76, 67,  2, 36, 69,
        70, 64, 62, 68, 76, 13,  0, 49, 79, 70, 81, 87, 72, 66, 79,  2, 52, 64,
      

# Let's now split up the data into train and validation sets

In [63]:

n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [64]:
block_size = 8
train_data[:block_size+1]

tensor([42,  0,  0, 42, 42,  0,  0, 42, 42])

# Generate Context-Target Pairs for Training

In [65]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([42]) the target: 0
when input is tensor([42,  0]) the target: 0
when input is tensor([42,  0,  0]) the target: 42
when input is tensor([42,  0,  0, 42]) the target: 42
when input is tensor([42,  0,  0, 42, 42]) the target: 0
when input is tensor([42,  0,  0, 42, 42,  0]) the target: 0
when input is tensor([42,  0,  0, 42, 42,  0,  0]) the target: 42
when input is tensor([42,  0,  0, 42, 42,  0,  0, 42]) the target: 42


# Batch Generation and Context-Target Pair Display

In [66]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[20, 13,  2, 18, 26, 26, 20, 15],
        [86,  0, 62, 64, 81, 70, 83, 62],
        [80, 74, 76, 72, 66, 15,  2, 42],
        [73, 86,  2, 77, 69, 62, 80, 66]])
targets:
torch.Size([4, 8])
tensor([[13,  2, 18, 26, 26, 20, 15,  0],
        [ 0, 62, 64, 81, 70, 83, 62, 81],
        [74, 76, 72, 66, 15,  2, 42, 75],
        [86,  2, 77, 69, 62, 80, 66,  2]])
----
when input is [20] the target: 13
when input is [20, 13] the target: 2
when input is [20, 13, 2] the target: 18
when input is [20, 13, 2, 18] the target: 26
when input is [20, 13, 2, 18, 26] the target: 26
when input is [20, 13, 2, 18, 26, 26] the target: 20
when input is [20, 13, 2, 18, 26, 26, 20] the target: 15
when input is [20, 13, 2, 18, 26, 26, 20, 15] the target: 0
when input is [86] the target: 0
when input is [86, 0] the target: 62
when input is [86, 0, 62] the target: 64
when input is [86, 0, 62, 64] the target: 81
when input is [86, 0, 62, 64, 81] the target: 70
when input is [86, 0,

In [67]:
print(xb) # our input to the transformer

tensor([[20, 13,  2, 18, 26, 26, 20, 15],
        [86,  0, 62, 64, 81, 70, 83, 62],
        [80, 74, 76, 72, 66, 15,  2, 42],
        [73, 86,  2, 77, 69, 62, 80, 66]])


# Define and Test a Bigram Language Model in PyTorch

In [68]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


torch.Size([32, 123])
tensor(5.3819, grad_fn=<NllLossBackward0>)

§‡gB½<>S∆0fÜ™
½[■2l=—©</½Qe–■NY±haζYN≥eÜ>6—–bnl∼Cs?Nd½sα●5ü*@TvGc±xZx!ggcUü7ÜaZfg–ˆ αI'q)pO™©PT(gA


In [69]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [70]:
batch_size = 32
for steps in range(100): # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())


5.285351753234863


In [71]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


5O∆→2‡*zGWXB]m5ü0]A%ZqByK@O<psa@BÜ;&ïx8x<U—kr;O—‡ühE●é!λ<A3yE@XápÜPκSKδuAHu½∆Q12lXái+×!ZuS pö,n×?3α½Vλü<EïVlvuSYi;I•ET$h$©.k27—lX†7EloJ
>±α'L2Kh;O™KδnüZUjá&3ÜR
=N;x,HxO7*d7rCXtïSeZ-z)pp5,)I≥‡Uf'F×>7)λζ™δβ p$p—H—×=P)™†?≤4(Rj.×;†Q*q©]e!S8<rq[—→á!Sc-[/,arr
5bi@NH2o@½N-X,?Qj†ˆ×CUu,>uepö≥>EWnfygu4v½tGEE–J(ζRÜµokcβü'Fγfpx?"α&κ@U≥]*κδ8)V‡
)δho—†MxO>r0k;>.8'o@Ul;—λfBgAh0v Nn+■2zeb—Yi'F]D)H*–pGhJlVMαV†©jA ,[[Sκ@.:×?ET3s≥ÜTÜ½•MiVüa±G½j4,E½(.I≥±HtH§?;IB>i•eXm,ï.9L'cc×yo)L•αö;9H[■∆ˆ ∼h™PgRPé≥:;‡?x


## The mathematical trick in self-attention

In [72]:
# toy example illustrating how matrix multiplication can be used for a "weighted aggregation"
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [73]:
# consider the following toy example:

torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [74]:
# We want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)


In [75]:
# version 2: using matrix multiply for a weighted aggregation
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)
torch.allclose(xbow, xbow2)

False

In [76]:
# version 3: use Softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)


False

In [77]:
# version 4: self-attention!
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)

# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)
wei =  q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v
#out = wei @ x

out.shape

torch.Size([4, 8, 16])

In [78]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)

Notes:
- Attention is a **communication mechanism**. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights.
- There is no notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode tokens.
- Each example across batch dimension is of course processed completely independently and never "talk" to each other
- In an "encoder" attention block just delete the single line that does masking with `tril`, allowing all tokens to communicate. This block here is called a "decoder" attention block because it has triangular masking, and is usually used in autoregressive settings, like language modeling.
- "self-attention" just means that the keys and values are produced from the same source as queries. In "cross-attention", the queries still get produced from x, but the keys and values come from some other, external source (e.g. an encoder module)
- "Scaled" attention additional divides `wei` by 1/sqrt(head_size). This makes it so when input Q,K are unit variance, wei will be unit variance too and Softmax will stay diffuse and not saturate too much. Illustration below

In [79]:
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-2, -1) * head_size**-0.5

In [80]:
k.var()

tensor(1.0449)

In [81]:
q.var()

tensor(1.0700)

In [82]:
wei.var()

tensor(1.0918)

In [83]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1)

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])

In [84]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5])*8, dim=-1) # gets too peaky, converges to one-hot

tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])

In [85]:
class LayerNorm1d: # (used to be BatchNorm1d)

  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)

  def __call__(self, x):
    # calculate the forward pass
    xmean = x.mean(1, keepdim=True) # batch mean
    xvar = x.var(1, keepdim=True) # batch variance
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    return self.out

  def parameters(self):
    return [self.gamma, self.beta]

torch.manual_seed(1337)
module = LayerNorm1d(100)
x = torch.randn(32, 100) # batch size 32 of 100-dimensional vectors
x = module(x)
x.shape

torch.Size([32, 100])

In [86]:
x[:,0].mean(), x[:,0].std() # mean,std of one feature across all batch inputs

(tensor(0.1469), tensor(0.8803))

In [87]:
x[0,:].mean(), x[0,:].std() # mean,std of a single input from the batch, of its features

(tensor(-9.5367e-09), tensor(1.0000))

In [88]:
# French to English translation example:

# <--------- ENCODE ------------------><--------------- DECODE ----------------->
# les réseaux de neurones sont géniaux! <START> neural networks are awesome!<END>



### Full finished code, for reference

You may want to refer directly to the git repo instead though.

In [89]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

# load and process data
torch.manual_seed(1337)


# !wget https://github.com/Nihar1402-iit/LLM/blob/main/Robbins-Pathologic2005.txt -O Robbins-Pathologic2005.txt
with open('Robbins-Pathologic2005.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

# Define Loss Estimation Function
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

# Define Model Components
##Self-Attention Head
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out
## Multi-Head Attention
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out
## Feed Forward
class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

## Transformer Block
class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

# Model Initialization:
model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# Training Loop:
for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))


0.217211 M parameters
step 0: train loss 4.9436, val loss 4.9391
step 100: train loss 2.8786, val loss 2.9379
step 200: train loss 2.6867, val loss 2.7197
step 300: train loss 2.6299, val loss 2.6502
step 400: train loss 2.5745, val loss 2.5959
step 500: train loss 2.5265, val loss 2.5497
step 600: train loss 2.4786, val loss 2.5002
step 700: train loss 2.4227, val loss 2.4568
step 800: train loss 2.4073, val loss 2.4340
step 900: train loss 2.3419, val loss 2.3765
step 1000: train loss 2.3051, val loss 2.3326
step 1100: train loss 2.2618, val loss 2.3101
step 1200: train loss 2.2390, val loss 2.2600
step 1300: train loss 2.1964, val loss 2.2462
step 1400: train loss 2.1840, val loss 2.2233
step 1500: train loss 2.1436, val loss 2.1889
step 1600: train loss 2.1346, val loss 2.1716
step 1700: train loss 2.0937, val loss 2.1510
step 1800: train loss 2.0926, val loss 2.1474
step 1900: train loss 2.0510, val loss 2.1010
step 2000: train loss 2.0409, val loss 2.0955
step 2100: train loss 2.